In [1]:
import os

In [2]:
%pwd

'c:\\Users\\DELL\\Projects\\ML-Mlflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\DELL\\Projects\\ML-Mlflow'

In [5]:
# 4. Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
#Config Manager
from mlflow.constants import *
from mlflow.utils.common import read_yaml, create_directories

In [7]:
# 5.Config Manager and src

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        param_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.param = read_yaml(param_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [8]:
# 6.Components
# import the packages

import os
import urllib.request as request
import zipfile
from mlflow import logger
from mlflow.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{header}")
        else:
            logger.info(f"file already exist of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        function returns None 
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
# 7 Pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-02-13 10:43:33,757: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-13 10:43:33,766: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-13 10:43:33,768: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-13 10:43:33,775: INFO: common: created directory at: artifacts]
[2025-02-13 10:43:33,776: INFO: common: created directory at: artifacts/data_ingestion]
[2025-02-13 10:43:35,982: INFO: 1394977229: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 22171
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "3414df89a76e1df22d910aef5e72f20096618198bbc9ac6a8ab17bdd99f53143"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: BCC0:28E173:230299:2DCE80:67AD7F79
Accept-Ranges: bytes
Date: Thu, 13